# Sports Betting LLM Project

This project demonstrates a Retrieval-Augmented Generation (RAG) pipeline using a Large Language Model (LLM). User queries are transformed into embeddings and compared against a vector database of domain-specific documents. The most relevant context is retrieved and supplied to the LLM, which generates a context-aware response.

In [1]:
from sentence_transformers import SentenceTransformer
# documents to be used for model context.
# These documents were created summarising FAQ responses found at bet365 here: https://help.bet365.com/en/technical-support/faqs
docs = [
    # Technical Support
    "Technical issues on the website may include display problems, deposit/withdrawal issues, login failures, unusual error messages, or unresponsive links. Bet365 provides 24/7 Customer Technical Support to help troubleshoot these issues.",

    # Slow Network or Device Performance
    "For time-critical events such as online poker, slower network speeds or device performance may put users at a disadvantage. Users should ensure their connection and device are optimal.",

    # Assistance for Visual Impairment
    "If you have difficulty viewing pages due to visual impairment, Bet365 encourages you to contact support for assistance.",

    # Online Security - TLS
    "Bet365 uses TLS encryption on all website pages to secure personal and payment information. The padlock symbol in the browser indicates that your data is encrypted.",

    # Online Security - Firewalls
    "Bet365 employs multiple firewalls to prevent unauthorized access to its servers and protect sensitive information.",

    # Online Security - Extra Measures
    "Bet365 operates an ISO/IEC 27001 certified Information Security Management System (ISMS) to ensure confidentiality, integrity, and availability of information. Users can visit www.getsafeonline.org for tips to stay secure online.",

    # Game Won’t Load
    "If a game fails to launch, browser cookie settings may be the cause. Adjust settings in Safari or Chrome to allow cookies, clear browsing data, and reopen the browser to access games.",

    # Waiting to Access App or Website
    "During high traffic periods, Bet365 may limit access and place users in a virtual queue with estimated wait times. The page automatically refreshes, and users will gain access once available.",

    # Safe Sender List
    "To receive important notifications, add Bet365 to your Safe Sender list. Emails are sent from support-eng@customerservice365.com, support-eng@365customerservice.com, and support-eng@customerservices365.com. Promotional emails include an unsubscribe option.",

    # Keep Me Logged In
    "The 'Keep me logged in' feature allows users to access their account without entering login details each time. To enable, select 'Keep me logged in' on the login popup and enter credentials. To disable, log out and re-enter credentials on next access."
]

# Load the pretrained sentence transformer model
# all-MiniLM-L6-v2 is a model taken from here: https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
# It maps sentences to a 384-dimensional vector space.
model = SentenceTransformer('all-MiniLM-L6-v2')

# Calculate the embeddings for the new documents
embeddings = model.encode(docs)
print(embeddings.shape)
# Check the similarity between document embeddings
similarity = model.similarity(embeddings,embeddings)
print(similarity)

(10, 384)
tensor([[1.0000, 0.3957, 0.6836, 0.6975, 0.6482, 0.6212, 0.2913, 0.6395, 0.6034,
         0.0567],
        [0.3957, 1.0000, 0.2532, 0.2427, 0.3728, 0.2448, 0.1973, 0.5112, 0.1304,
         0.1327],
        [0.6836, 0.2532, 1.0000, 0.5284, 0.4590, 0.4468, 0.2310, 0.4907, 0.5121,
         0.0721],
        [0.6975, 0.2427, 0.5284, 1.0000, 0.7482, 0.6784, 0.1456, 0.5695, 0.5528,
         0.1323],
        [0.6482, 0.3728, 0.4590, 0.7482, 1.0000, 0.7185, 0.1578, 0.6741, 0.5619,
         0.1942],
        [0.6212, 0.2448, 0.4468, 0.6784, 0.7185, 1.0000, 0.0202, 0.5141, 0.5960,
         0.0451],
        [0.2913, 0.1973, 0.2310, 0.1456, 0.1578, 0.0202, 1.0000, 0.1994, 0.0564,
         0.1962],
        [0.6395, 0.5112, 0.4907, 0.5695, 0.6741, 0.5141, 0.1994, 1.0000, 0.4282,
         0.2323],
        [0.6034, 0.1304, 0.5121, 0.5528, 0.5619, 0.5960, 0.0564, 0.4282, 1.0000,
         0.0370],
        [0.0567, 0.1327, 0.0721, 0.1323, 0.1942, 0.0451, 0.1962, 0.2323, 0.0370,
         1.0000]])

In [131]:
import torch

#input query and create an embedding for it.
query = "why wont my game load?"
query_embedding = model.encode(query)

#use cosine similarity to find which document embedding is most similar to the query embedding.
cos_scores = util.cos_sim(query_embedding, embeddings)
top_result = torch.argmax(cos_scores)
print("The most relevant document:", docs[top_result])


The most relevant document: If a game fails to launch, browser cookie settings may be the cause. Adjust settings in Safari or Chrome to allow cookies, clear browsing data, and reopen the browser to access games.


In [134]:
#generate an LLM output using the user prompt and the context from the documents.
from transformers import pipeline
# https://huggingface.co/google/flan-t5-large
#text2text takes a text prompt and outputs text using the selected model
generator = pipeline("text2text-generation", model="google/flan-t5-large")

context = docs[top_result]
prompt = f"You are a customer assistant chatbot. Based on this context: {context}\n Answer the following qustion in a full sentence with correct grammar:{query}\n start each sentence with a capital letter"

#generate an output based on the prompt 
answer = generator(prompt, max_length=200, do_sample=True)
print(answer[0]['generated_text'])


Device set to use cpu


If a game fails to launch, browser cookie settings may be the cause. Adjust settings in Safari or Chrome to allow cookies, clear browsing data, and reopen the browser to access games.
